In [1]:
#hide
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
#hide
%load_ext autoreload
%autoreload 2

%matplotlib inline
%cd /content/gdrive/My Drive/Colab Notebooks

/content/gdrive/My Drive/Colab Notebooks


In [ ]:
__all__ ='convT_norm_relu pad_conv_norm_relu resnet_generator conv_norm_leaky discriminator CycleGAN'.split(" ")

In [ ]:
#hide
__all__

['convT_norm_relu',
 'pad_conv_norm_relu',
 'resnet_generator',
 'conv_norm_leaky',
 'discriminator',
 'CycleGAN']

In [ ]:
#hide
# !git clone https://github.com/prajwal-suresh13/dl_lib.git

In [ ]:
from dl_lib.core.all import *

# Generator

In [ ]:
def convT_norm_relu(ch_in, ch_out, norm_layer, kernel_size=3, stride=2, bias=True, init=nn.init.kaiming_normal_):
    convT = nn.ConvTranspose2d(ch_in, ch_out, kernel_size=kernel_size, stride=stride, padding=1, output_padding=1, bias=bias)
    if init:
        init(convT.weight)
        if hasattr(convT, 'bias') and hasattr(convT.bias,'data'): convT.bias.data.fill_(0.)
    layers = [convT, norm_layer(ch_out), nn.ReLU(inplace=True)]
    return layers

In [ ]:
def pad_conv_norm_relu(ch_in, ch_out, pad_mode, norm_layer, kernel_size=3, bias=True, pad=1, stride=1, activ=True, init=nn.init.kaiming_normal_):
    layers = []
    if pad_mode == 'reflection': layers.append(nn.ReflectionPad2d(pad))
    elif pad_mode == 'border':   layers.append(nn.ReplicationPad2d(pad))
    p= pad if pad_mode =='zeros' else 0
    conv = nn.Conv2d(ch_in, ch_out, kernel_size=kernel_size, padding=p, stride=stride, bias=bias)
    if init:
        init(conv.weight)
        if hasattr(conv, 'bias') and hasattr(conv.bias,'data'): conv.bias.data.fill_(0.)
    layers += [conv, norm_layer(ch_out)]
    if activ: layers.append(nn.ReLU(inplace=True))
    return layers

In [ ]:
class ResnetBlock(nn.Module):
    def __init__(self, dim, pad_mode='reflection', norm_layer=None, dropout=0., bias=True):
        super().__init__()
        assert pad_mode in ['zeros','reflection','border'], f'padding {pad_mode} not implemented'
        norm_layer = (nn.InstanceNorm2d if norm_layer is None else norm_layer)
        layers = pad_conv_norm_relu(dim, dim, pad_mode, norm_layer, bias=bias)
        if dropout!=0: layers.append(nn.Dropout(dropout))
        layers += pad_conv_norm_relu(dim, dim, pad_mode, norm_layer, bias=bias, activ=False)
        self.conv_block = nn.Sequential(*layers)

    def forward(self, x):return x + self.conv_block(x)

In [ ]:
def resnet_generator(ch_in, ch_out, n_filters=64, norm_layer=None, dropout=0, n_blocks=9, pad_mode='reflection'):
    norm_layer = (nn.InstanceNorm2d if norm_layer is None else norm_layer)
    bias = (norm_layer == nn.InstanceNorm2d)
    layers = pad_conv_norm_relu(ch_in, n_filters, 'reflection', norm_layer, kernel_size=7, pad=3, bias=bias)
    for i in range(2):
        layers += pad_conv_norm_relu(n_filters, n_filters*2, 'zeros', norm_layer, stride=2, bias=bias)
        n_filters *= 2
    layers += [ResnetBlock(n_filters, pad_mode, norm_layer, dropout, bias) for _ in range(n_blocks)]
    for i in range(2):
        layers += convT_norm_relu(n_filters, n_filters//2, norm_layer, bias=bias)
        n_filters//=2
    layers += [nn.ReflectionPad2d(3), nn.Conv2d(n_filters, ch_out, kernel_size=7, padding=0), nn.Tanh()]
    return nn.Sequential(*layers)

In [ ]:
#hide
model = resnet_generator(3,3)
from torchsummary import summary
summary(model,(3,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ReflectionPad2d-1          [-1, 3, 134, 134]               0
            Conv2d-2         [-1, 64, 128, 128]           9,472
    InstanceNorm2d-3         [-1, 64, 128, 128]               0
              ReLU-4         [-1, 64, 128, 128]               0
            Conv2d-5          [-1, 128, 64, 64]          73,856
    InstanceNorm2d-6          [-1, 128, 64, 64]               0
              ReLU-7          [-1, 128, 64, 64]               0
            Conv2d-8          [-1, 256, 32, 32]         295,168
    InstanceNorm2d-9          [-1, 256, 32, 32]               0
             ReLU-10          [-1, 256, 32, 32]               0
  ReflectionPad2d-11          [-1, 256, 34, 34]               0
           Conv2d-12          [-1, 256, 32, 32]         590,080
   InstanceNorm2d-13          [-1, 256, 32, 32]               0
             ReLU-14          [-1, 256,

# Discriminator

In [ ]:
def conv_norm_leaky(ch_in, ch_out, norm_layer=None, kernel_size=3, bias=True, pad=1, stride=1, activ=True, slope=0.2, init=nn.init.kaiming_normal_):
    conv = nn.Conv2d(ch_in, ch_out, kernel_size=kernel_size, padding=pad, stride=stride, bias=bias)
    if init:
        init(conv.weight)
        if hasattr(conv, 'bias') and hasattr(conv.bias, 'data'): conv.bias.data.fill_(0)
    layers = [conv]
    if norm_layer is not None: layers.append(norm_layer(ch_out))
    if activ: layers.append(nn.LeakyReLU(slope,inplace=True))
    return layers

In [ ]:
def discriminator(ch_in, n_filters=64, n_layers=3, norm_layer=None, sigmoid=False):
    norm_layer = (nn.InstanceNorm2d if norm_layer is None else norm_layer)
    bias = (norm_layer == nn.InstanceNorm2d)
    layers = conv_norm_leaky(ch_in, n_filters, kernel_size=4, stride=2, pad=1)
    for i in range(n_layers-1):
        new_filters = 2*n_filters if i<=3 else n_filters
        layers += conv_norm_leaky(n_filters, new_filters, norm_layer, kernel_size=4, stride=2, pad=1, bias=bias)
        n_filters = new_filters
    new_filters = 2*n_filters if n_layers <=3 else n_filters
    layers += conv_norm_leaky(n_filters, new_filters, norm_layer, kernel_size=4, stride=1, pad=1, bias=bias)
    layers.append(nn.Conv2d(new_filters, 1, kernel_size=4, stride=1, padding=1))
    if sigmoid: layers.append(nn.Sigmoid())
    return nn.Sequential(*layers)

In [ ]:
#hide
disc = discriminator(3)
summary(disc,(3,128,128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 64, 64]           3,136
         LeakyReLU-2           [-1, 64, 64, 64]               0
            Conv2d-3          [-1, 128, 32, 32]         131,200
    InstanceNorm2d-4          [-1, 128, 32, 32]               0
         LeakyReLU-5          [-1, 128, 32, 32]               0
            Conv2d-6          [-1, 256, 16, 16]         524,544
    InstanceNorm2d-7          [-1, 256, 16, 16]               0
         LeakyReLU-8          [-1, 256, 16, 16]               0
            Conv2d-9          [-1, 512, 15, 15]       2,097,664
   InstanceNorm2d-10          [-1, 512, 15, 15]               0
        LeakyReLU-11          [-1, 512, 15, 15]               0
           Conv2d-12            [-1, 1, 14, 14]           8,193
Total params: 2,764,737
Trainable params: 2,764,737
Non-trainable params: 0
---------------------------

In [ ]:
#hide
nn.Sequential(*flatten_model(resnet_generator(3,3)),*flatten_model(resnet_generator(3,3)))

Sequential(
  (0): ReflectionPad2d((3, 3, 3, 3))
  (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
  (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (3): ReLU(inplace=True)
  (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (9): ReLU(inplace=True)
  (10): ReflectionPad2d((1, 1, 1, 1))
  (11): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
  (12): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (13): ReLU(inplace=True)
  (14): ReflectionPad2d((1, 1, 1, 1))
  (15): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
  (16): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_

# CycleGAN model

In [ ]:
class CycleGAN(nn.Module):

    def __init__(self, ch_in, ch_out, n_features=64, disc_layers=3, gen_blocks=9, lsgan=True, drop=0., norm_layer=None):
        super().__init__()
        self.D_A = discriminator(ch_in, n_features, disc_layers, norm_layer, sigmoid = not lsgan)
        self.D_B = discriminator(ch_in, n_features, disc_layers, norm_layer, sigmoid = not lsgan)
        self.G_A = resnet_generator(ch_in, ch_out, n_features, norm_layer, drop, gen_blocks)
        self.G_B = resnet_generator(ch_in, ch_out, n_features, norm_layer, drop, gen_blocks)

    def forward(self, input):
        real_A, real_B = input
        fake_A, fake_B = self.G_A(real_B), self.G_B(real_A)
        cyc_A , cyc_B = self.G_A(fake_B), self.G_B(fake_A)
        idt_A, idt_B = self.G_A(real_A), self.G_B(real_B)
        return [fake_A, fake_B, cyc_A, cyc_B, idt_A, idt_B]

In [4]:
#hide
!pip install fire
!python dl_lib/notebook2script.py image_colorization/cyclegan/cycleganmodel.ipynb dl_lib/cyclegan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Converted image_colorization/cyclegan/cycleganmodel.ipynb to dl_lib/cyclegan/cycleganmodel.py
